In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../../measure/SAF/result/safety_hangang_total_score.csv")
nonhangang_df = pd.read_csv("../../../measure/SAF/result/safety_nonhangang_total_score.csv")

In [3]:
# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [4]:
# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 RST_score
gu = gu.merge(
    nonhangang_df[["자치구", "RST_score"]],
    left_on="SIGUNGU_NM",
    right_on="자치구",
    how="left"
).drop(columns=["자치구"])

# 한강공원: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)

# 한강공원 RST_score merge
parks = parks.merge(
    hangang_df[["한강공원", "RST_score"]],
    left_on="park_name",
    right_on="한강공원",
    how="left"
).drop(columns=["한강공원"])

In [5]:
gu

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry,RST_score
0,20240630,동대문구,11060,"MULTIPOLYGON (((127.06937 37.60828, 127.06938 ...",30.735480
1,20240630,중랑구,11070,"MULTIPOLYGON (((127.11131 37.62069, 127.11193 ...",29.765667
2,20240630,성북구,11080,"MULTIPOLYGON (((126.98396 37.63644, 126.9842 3...",54.071261
3,20240630,강북구,11090,"MULTIPOLYGON (((127.00459 37.68507, 127.00553 ...",31.861530
4,20240630,도봉구,11100,"MULTIPOLYGON (((127.01983 37.70102, 127.02215 ...",34.160672
5,20240630,노원구,11110,"MULTIPOLYGON (((127.08385 37.69387, 127.08388 ...",49.008933
6,20240630,은평구,11120,"MULTIPOLYGON (((126.94794 37.65838, 126.94792 ...",36.187918
7,20240630,서대문구,11130,"MULTIPOLYGON (((126.95262 37.60646, 126.95266 ...",18.625376
8,20240630,마포구,11140,"MULTIPOLYGON (((126.88936 37.58435, 126.88945 ...",19.014382
9,20240630,양천구,11150,"MULTIPOLYGON (((126.87405 37.54694, 126.87417 ...",42.674469


In [6]:
# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["RST_score"].min(), parks["RST_score"].min())
all_max = max(gu["RST_score"].max(), parks["RST_score"].max())

cmap = plt.cm.Oranges
norm = colors.Normalize(vmin=all_min, vmax=all_max)

In [7]:
# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["RST_score"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["RST_score"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "RST_score"],
        aliases=["자치구", "휴식 점수"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["RST_score"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["RST_score"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "RST_score"],
        aliases=["공원", "휴식 점수"]
    )
).add_to(m)

In [8]:
# 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("../../result/SAF/rst_score_visualization.html")
print("rst_score_visualization.html 저장 완료")

rst_score_visualization.html 저장 완료
